In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
# import openai
# from sentence_transformers import SentenceTransformer, util
import json
import csv
import re

### A. Preparing the datasets.

#### A.1 Deleting extra samples from each state in the sample_final_full file by picking randomly

In [1]:
def delete(file_from, states, numbers, file_to):
    sample = pd.read_csv(file_from)
    for i in range(len(states)):
        to_delete = sample[sample['us_state']==states[i]].sample(n = numbers[i])
        print(states[i])
        print(to_delete.shape[0])
        sample = sample[~sample['google_id'].isin(to_delete['google_id'].to_list())]
#         sample.drop(sample.google_id in to_delete['google_id'].to_list())
        print(sample.shape[0])
    sample.to_csv(file_to, index=False, encoding='utf-8')

numbers=[5,4,3,3,3,2,2,2,2,2,2,2,1,1,1,1,1,1,1]
states=['Oregon','Idaho','Montana','New Hampshire','Rhode Island','Arizona','District Of Columbia','Hawaii','Louisiana','Mississippi','Nevada','Wyoming','Colorado','Delaware','Maine','Nebraska','New Mexico','South Dakota','Vermont']
delete('output/sample_final-full.csv',states,numbers,'output/sample_no_duplicates_1.csv')

NameError: name 'pd' is not defined

#### A.2 Here we take the sample and returns a filtered sample with restaurants that have both Chinese and English names

In [7]:
def get_Chinese(file_from, file_to):
    sample = pd.read_csv(file_from)
    sample_Chinese = sample.dropna(subset=['Chinese_Name'])
    sample_Chinese.to_csv(file_to,index=False,encoding='utf-8')

get_Chinese('output/sample_no_duplicates.csv','output/sample_parallel.csv')

#### A.3. detecting duplicates in the parallel cleaned sample

In [10]:
sample = pd.read_csv('output/sample_no_duplicates.csv')
# sample = sample[sample.google_id != '0x80dc9e0db4134fb9:0x117c553fca7a0c1']
# sample = sample[sample.google_id != '0x809539b3cbe32de1:0x823a3dd6e4ae484c']
# sample.to_csv('output/sample_no_duplicates_1.csv',index=False,encoding='utf-8')
sample = sample.dropna(subset=['Chinese_Name'])
duplicate = sample[sample.duplicated(['Chinese_Name','English_Name'])]
print(duplicate)
duplicate.to_csv('others/double_duplicates.csv')

#### A.4. Deleting double duplicates. After manually checking the dataset, we found that some data picked out by pandas are not double duplicates, so we had to remove those datapoints by hand.

In [7]:
sample_parallel = pd.read_csv('output/sample_cleaned.csv')
duplicates = pd.read_csv('utilities/double_duplicates_revised.csv')
delete = sample_parallel.google_id.isin(duplicates.google_id)
sample_parallel['delete'] = delete
sample_parallel = sample_parallel[sample_parallel.delete == False]
sample_parallel.to_csv('output/sample_no_duplicates.csv',index=False, encoding='utf-8')

#### A.5. Helper function: gets state frequency count in a dataset

In [3]:
# this function focuses on a single column and returns the frequencies of the values in a csv
def get_freq(file, column,save_to):
    file = pd.read_csv(file)
    counts = {}
    for value in file[column]:
        counts[value] = counts.get(value, 0) + 1
    with open(save_to,'w') as f:
        w = csv.writer(f)
        w.writerows(counts.items())

get_freq('output/national.csv','us_state','others/national_freq.csv')
get_freq('sample_final_deduplcated_full.csv','us_state','others/state_freq.csv')


/var/folders/ck/1xc3_hrd4556zhtcl0bbnq9r0000gn/T/ipykernel_1269/324090569.py:3: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv(file)


#### A.6 Formatting datatypes of certain columns

In [6]:
# def correct_dtype(file_from, cols, dtype, file_to):
#     df = pd.read_csv(file_from)
#     # for column in cols:
#     #     # try:
#     #     # df[column] = df[column].astype(dtype)
#     #     df[[column]] = df[[column]].apply(pd.to_numeric)
#     #     print(df[column].dtype)
#     df = df.convert_dtypes()
#     df.to_csv(file_to, index=False, encoding = 'utf-8')

def find_value(google_id, target_ind, dfs):
    target_new_value = 0
    for df in dfs:
        # print(df['google_id'].to_list())
        if google_id in df['google_id'].to_list():
            
            print('found')
            # print(df.loc[df['google_id'] == google_id])
            # target_new_value = df.loc[df['google_id'] == google_id][target_ind]
            ### try:
            target_new_value = df.loc[df['google_id'] == google_id].values[0][-2]

            ###################################################
            # target_new_value = target_new_value.to_string()
            # print(df.loc[df['google_id'] == google_id])
            print(target_new_value)
    return target_new_value

def apply_value(file_from,file_to, column, dfs):
    # read csv
    df = pd.read_csv(file_from)
    target = df['google_id'].apply(find_value, args=(column, dfs)) 
    df[column+'_new'] = target # add new column to dataframe
    df.to_csv(file_to,index=False, encoding='utf-8')
    # return df

# TODO1: use the rawest data on google drive instead of the ones here.
# TODO2: need to figure out how to extract value from series
files = ['east_coast_1.csv','east_coast_2.csv','mountain_west.csv','south.csv','pacific_west.csv','midwest.csv']
dfs = []
for file in files:
        df = pd.read_csv('raw_datasets/'+file)
        # print(df['cid'].dtype)
        # df['cid'] = df['cid'].astype('string')
        # df['reviews_id'] = df['reviews_id'].astype('string',errors='ignore')
        # df['reviews_id'] = df['reviews_id'].astype('int64',errors='ignore')
        dfs.append(df)
apply_value('output/sample_final_full.csv', 'output/national_testing.csv','reviews_id',dfs)
# correct_dtype('output/sample_final_full.csv',['cid','reviews_id'],'int64Dtype','output/sample_final_full_dtype_corrected.csv')

/var/folders/ck/1xc3_hrd4556zhtcl0bbnq9r0000gn/T/ipykernel_1065/1328555454.py:42: DtypeWarning: Columns (20,42,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_datasets/'+file)
/var/folders/ck/1xc3_hrd4556zhtcl0bbnq9r0000gn/T/ipykernel_1065/1328555454.py:42: DtypeWarning: Columns (20,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_datasets/'+file)


found
-1022523433966798688
found
-4396075968227768531
found
7573336596774913118
found
3973210082516602585
found
3973210082516602585
found
3372350369733266509
found
5604783954543597335
found
-2700452531948514729
found
5614687129442117912
found
-1935209102414919723
found
6919344840819657314
found
-7739734298085959682
found
1895539680791801654
found
5001193609535563706
found
-3606794679412686961
found
6283839175388366385
found
-6505917703701565356
found
-4485748287454265540
found
7899956302000481203
found
7899956302000481203
found
8014579175988872319
found
8014579175988872319
found
3528861890635453577
found
-4123596961390847227
found
2248716039789712392
found
-2142161285620625993
found
-2142161285620625993
found
367076457271785000
found
8017058768459667645
found
5189338882406732784
found
-6413042361372966287
found
-2146519700628827674
found
-657718403568796901
found
-3830224870017210734
found
-2825442740144150153
found
-5295614349924699664
found
-4401065171420107053
found
-604512134174705

### B. Helper function for merging two datasets

In [7]:
# this function merges guest dataframe into host dataframe. Method is the how argument in the pandas merge method
# and column_name is the on argument in the pandas merge method.
def merge(host_file, guest_file, method, column_name, save_file):
    host = pd.read_csv(host_file)
    guest = pd.read_csv(guest_file)
    host = host.merge(guest, how = method, on = column_name)
    host = host.dropna(subset=['google_id'])
    host.to_csv(save_file, index=False, encoding='utf-8')

## to be executed
merge('utilities/labeled_datasets/cn_golden_label.csv','output/sample_final_parallel.csv','left','national_id','output/cn_golden_label.csv')
merge('utilities/labeled_datasets/en_golden_label.csv','output/sample_final_parallel.csv','left','national_id','output/en_golden_label.csv')

### C. Splitting each labeled dataset into validation and testing sets

#### C.1 Get rid of duplicate names and split a dataset into training and validation sets

In [3]:
# helper function. This function drops duplicate items in a column
def get_unique(df, col_name):
    return df.drop_duplicates(subset=[col_name])

# helper function
def remove_overlaps(df,df2):
        # removing rows in df2 that appeared in df, resulting in a filtered df
        df_new = pd.merge(df,df2, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
        return df_new

def split(df, col_name, training_filename, validation_filename):
    df = get_unique(df, col_name)
    training = df.sample(frac=0.5, random_state=1)
    validation = remove_overlaps(df, training)
    training.to_csv(training_filename, index=False, encoding='utf-8')
    validation.to_csv(validation_filename, index=False, encoding='utf-8')

# to be executed
golden_cn = pd.read_csv('output/cn_golden_label.csv')
golden_en = pd.read_csv('output/en_golden_label.csv')
split(golden_cn, 'Chinese_Name', 'output/validation_cn.csv','output/test_cn.csv')
split(golden_en, 'English_Name', 'output/validation_en.csv','output/test_en.csv')

#### C.2 Getting information of testing and validadtion datasets

In [3]:
def get_shape(file_name):
    sample = pd.read_csv('output/'+file_name+'.csv')
    print(file_name+': '+str(sample.shape[0]))

for item in ['test_cn','test_en','validation_cn','validation_en']:
    get_shape(item)

test_cn: 363
test_en: 400
validation_cn: 363
validation_en: 400
